In [1]:
import torch
import torch.utils.data.dataloader as dataloader
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch.optim as optim
import os
import re
import sys
import gc
import gensim
import numpy as np
from tqdm import tqdm
from collections import Counter
from itertools import chain

In [2]:
text = []
for file in os.listdir('Holmes_Training_Data/'):
    with open(os.path.join('Holmes_Training_Data', file), 'r', errors='ignore') as f:
        text.extend(f.read().splitlines())

text = [x.replace('*', '') for x in text]
text = [x for x in text if x != '']
text = [re.findall(r"[\w']+|[.,!?;]", x) for x in text]
[x.insert(0, '<bos>') for x in text]
[x.insert(len(x), '<eos>') for x in text]
text = text[:10000]

In [3]:
vocab = set(chain(*text))
vocab_size = len(vocab)
freqs = Counter(chain(*text))

In [4]:
word_to_idx = {word: i+1 for i, word in enumerate(vocab)}
word_to_idx['<unk>'] = 0
idx_to_word = {i+1: word for i, word in enumerate(vocab)}
idx_to_word[0] = '<unk>'

In [5]:
freqs_pow = torch.Tensor([freqs[idx_to_word[i]] for i in range(vocab_size)]).pow(0.75)
dist = freqs_pow / freqs_pow.sum()
gc.collect()

0

In [6]:
def neg_sample(num_samples, batch_size, use_gpu):
    w = np.random.choice(len(dist), (batch_size, num_samples), p=dist.numpy())
    if use_gpu:
        return torch.tensor(w).to(device)
    else:
        return torch.tensor(w)

In [7]:
wvmodel = gensim.models.KeyedVectors.load_word2vec_format('test_word.txt',
                                                          binary=False, encoding='utf-8')

In [8]:
class seqNet(nn.Module):
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers,
                 bidirectional, weight, num_labels, use_gpu, **kwargs):
        super(seqNet, self).__init__(**kwargs)
        self.num_hiddens = num_hiddens
        self.num_layers = num_layers
        self.use_gpu = use_gpu
#         self.num_negs = num_negs
        self.bidirectional = bidirectional
        self.embedding = nn.Embedding.from_pretrained(weight)
        self.embedding.weight.requires_grad = False
        self.front = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                             num_layers=num_layers, bidirectional=self.bidirectional,
                             dropout=0)
        self.back = nn.LSTM(input_size=embed_size, hidden_size=self.num_hiddens,
                            num_layers=num_layers, bidirectional=self.bidirectional,
                            dropout=0)
        if self.bidirectional:
            self.decoder = nn.Linear(num_hiddens * 4, num_labels)
        else:
            self.decoder = nn.Linear(num_hiddens * 2, num_labels)

    def forward(self, inputs):
        embeddings_front = self.embedding(inputs[0])
        embeddings_back = self.embedding(inputs[1])
        states_front, _ = self.front(embeddings_front.permute([1, 0, 2]))
        states_back, _ = self.back(embeddings_back.permute([1, 0, 2]))
        encoding = torch.cat([states_front[-1], states_back[-1]], dim=1)
        outputs = self.decoder(encoding)
        return(outputs)

In [9]:
embedding_dim = 100
hidden_dim = 100
lr = 0.1
num_epoch = 500
use_gpu = True
num_layers = 2
bidirectional = True
batch_size = 256
num_negs = 25
device = torch.device('cuda:1')
loss_function = nn.CrossEntropyLoss()

In [10]:
def encode_sample(seq, word_to_idx):
    feature = []
    for token in seq:
        feature.append(word_to_idx[token])
    return feature

In [11]:
def padding(seq, maxlen, pad='<unk>'):
    padded = []
    if len(seq) < maxlen:
        padded = seq
        while len(padded) < maxlen:
            padded.append(pad)
    else:
        padded = seq[:maxlen]
    return padded

In [12]:
def eval_acc(y_pred, y_true):
    return torch.sum(torch.argmax(y_pred, dim=1) == y_true).cpu().numpy() / len(y_true)

In [13]:
data = []
for i in range(len(text)):
    for j in range(1, len(text[i])-1):
        context = []
        context.append(text[i][:j])
        context.append(text[i][(j+1):])
        target = word_to_idx[text[i][j]]
        context[0] = padding(context[0], 10)
        context[0] = encode_sample(context[0], word_to_idx)
        context[1] = padding(context[1], 10)
        context[1] = encode_sample(context[1], word_to_idx)
        data.append((context, target))

In [14]:
weight = nn.Embedding(len(word_to_idx), embedding_dim).weight.data
for i in range(len(wvmodel.index2word)):
    try:
        index = word_to_idx[wvmodel.index2word[i]]
    except:
        continue
    weight[index, :] = torch.from_numpy(wvmodel.get_vector(
        idx_to_word[word_to_idx[wvmodel.index2word[i]]]))

In [15]:
model = seqNet(vocab_size=len(word_to_idx), embed_size=embedding_dim,
               num_hiddens=hidden_dim, num_layers=num_layers,
               bidirectional=bidirectional, weight=weight, num_labels=len(word_to_idx),
               use_gpu=use_gpu)
optimizer = optim.SGD(model.parameters(), lr=0.1)
if use_gpu:
    model.to(device)

In [16]:
data_iter = torch.utils.data.DataLoader(data, batch_size=batch_size,
                                        shuffle=False, num_workers=0)

In [20]:
test_target = 'of'
test = [[], []]
test[0] = padding(['<bos>', 'The', 'Project', 'Gutenberg', 'Etext'], 10)
test[0] = encode_sample(test[0], word_to_idx)
test[1] = padding(['Reminiscences', 'of', 'Tolstoy', '<eos>'], 10)
test[1] = encode_sample(test[1], word_to_idx)
test = torch.tensor(test).view(2, 1, 10)
idx_to_word[int(torch.argmax(model(test.to(device)), dim=1).cpu().data)]

'avail'

In [21]:
for epoch in range(num_epoch):
    total_loss = torch.Tensor([0])
    num = 0
    train_acc = 0
    for context, target in tqdm(data_iter):
        num += 1
        context[0] = torch.stack(context[0]).permute((1, 0))
        context[1] = torch.stack(context[1]).permute((1, 0))
        context = torch.stack(context)
        target = torch.tensor(target)
        if use_gpu:
            context = context.to(device)
            target = target.to(device)
        model.zero_grad()
        log_probs = model(context)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        train_acc += eval_acc(log_probs, target)
    print('epoch %d, loss %.4f, acc %.4f' %(epoch, total_loss / num, train_acc / num))

  3%|▎         | 12/466 [00:00<00:03, 116.97it/s]

epoch 0, loss 7.4706, acc 0.0745


  3%|▎         | 12/466 [00:00<00:03, 113.75it/s]

epoch 1, loss 6.6842, acc 0.0763


  3%|▎         | 13/466 [00:00<00:03, 125.04it/s]

...

epoch 497, loss 0.4353, acc 0.9493


  3%|▎         | 12/466 [00:00<00:03, 116.99it/s]

epoch 498, loss 0.4886, acc 0.9313


100%|██████████| 466/466 [00:03<00:00, 116.91it/s]

epoch 499, loss 0.4671, acc 0.9380


In [22]:
idx_to_word[int(torch.argmax(model(test.to(device)), dim=1).cpu().data)]

'of'